In [2]:
import pandas as pd
data = pd.read_feather("dataFrame.feather")
print(data.shape)

(2225, 3)


In [ ]:
import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('/home/robertb/Desktop/text_embeddings/project/GoogleNews-vectors-negative300.bin', binary=True)  
